# IT Security - Sheet 3 "Asymmetric Crypography"

**Total achievable points: 20**

**Released: 21.11.2025**

**Submission Deadline: 28.11.2025 13:00**

---
Groupnumber: 17

Names and matriculation numbers of **ALL** team members: 

Neo Ahrens (456647)

Christian Bick (456513)

Yorck Heilmann (456599)

---

**Important Information**

The assignments have to be submitted by groups of 4 students. Even if you are registered in RWTHmoodle to a submission group, **please include the group number as well as the name and matriculation number of every group member in this notebook**. In case you are not part of a submission group and want to hand in assignments, please contact `ba-itsec@itsec.rwth-aachen.de`.

Enter your solutions for the tasks in the respective cells of this notebook. These cells are either marked by "YOUR ANSWER HERE" or `#YOUR CODE HERE`. Do not add any new cells or remove existing ones, especially do not copy cells. Cells marked with `###PLAYGROUND` can be used to test your implementation and generate output (see example for the first tasks). Do not add any other output or tests in the cell of the task, just implement the function with the header provided. If you want to test your implementation, use the `###PLAYGROUND` cells. They will be ignored during grading. **Do not change any other cells or add new ones.**

All assertions provided by us should be considered part of the exercise description. Passing the assertions does not automatically mean that you'll get full points for the corresponding task, but failing the assertions pretty much guarantees that you'll get no points. 

**Do not import any further Python packages** except the default Python ones and the ones that are explicitly given by us.

## 0. Setup (0 points)
*This assignment has one big storyline that spans Tasks 1-3, but all tasks can still be solved independently, e.g., you can solve Task 2 even if you have not solved Task 1.*

This exercise will use occasionally use the *cryptography* package. If you are using the same environment as for the previous exercise, you should have installed it already. Otherwise, you can likely install it by uncommenting and executing the cell below. Note that we will not provide tech support if the cell fails to install the package for whatever reason.

In [43]:
### PLAYGROUND
#import sys
#!{sys.executable} -m pip install cryptography

### Your new job at Sloplock

Good news: You have gotten a job offer as *Vibe-code cleanup specialist* at an up-and-coming IoT startup, specifically at their *Smart LOck Product* (SLOP) team. The team is lead by Prompty McBug, who is the reason why a *Vibe-code cleanup specialist* is needed in the first place. While it does seem like this will be a more "dynamic" work environment compared to the Agency (from the previous exercise), the offered money is just too good to say no. Hence, you accept the offer, and are stoked for the future ahead, but also a little bit scared.

On your first day, you are told just chat a little bit with the internal chat bot for onboarding. This was not very enjoyable, but you find out that your team is producing a smart lock that can only be opened and closed using an app, creatively called the *Sloplock*. The whole system is built arround a proprietary protocol, whose details still evade you, but the chatbot insists that it must be secure since it uses military-grade RSA.

![Sloplock logo; generated with AI](sloplock.png)
*Generated using AI*

## 1. RSA Fundamentals (5 points)

Unfortunately, your onboarding is cut short when Prompty McBug accidentally deletes half of the key generation scripts while trying to merge the repository using a single, overly verbose prompt to an outsourced LLM model. While he is able to find the error, he decides to just ask the LLM to fix it. The LLM then panics and deletes the whole Git repository. The only thing left is the half-finished code, that is able to generate primes `p` and `q`, and to find a suitable `e` as follows. 

In [44]:
def generate_prime(bit_size: int) -> int:
    # Generates a prime of bit-length bit_size.
    # This implementation is extremely cursed. You can see it as a testament 
    # to lengths I go so that you do not have to install another library.
    from cryptography.hazmat.primitives.asymmetric.rsa import generate_private_key
    return generate_private_key(2**16+1, 2*bit_size).private_numbers().p

def gcd(x: int, y: int) -> int:
    if x == 0:
        return y
    return gcd(y % x, x)

def get_e(p: int, q: int) -> int:
    phi_N = (p-1) * (q-1)
    for i in range(2, phi_N):
        if gcd(i, phi_N) == 1:
            return i

### Task 1.1 The extended Euclidean algorithm (2 points)

However, a crucial part is missing from the codebase: The calculating of the private exponent `d`! Seems like your first job is to fix that. What an eventfull first day at your new workplace...

From the lecture you know that you need the extended Euclidean algorithm to calculate the `d`. Given two integer inputs `a` and `b`, this algorithm computes two integer outputs `x` and `y` such that `x*a + b*y = gcd(a,b)`. Implement the algorithm in the function below, which should return the tuple `(x, y)`. **Pay attention to the order of the outputs!**

In [45]:
def extended_gcd(a: int, b: int) -> tuple[int, int]:
    # YOUR CODE HERE
    r_prev, r_curr = max(a,b), min(a,b)
    u_prev, u_curr = 1, 0
    v_prev, v_curr = 0, 1

    while r_curr != 0:
        quotient = r_prev // r_curr
        r_next = r_prev % r_curr
        
        u_next = u_prev - quotient * u_curr
        v_next = v_prev - quotient * v_curr
        
        r_prev, r_curr = r_curr, r_next
        u_prev, u_curr = u_curr, u_next
        v_prev, v_curr = v_curr, v_next
    
    return (u_prev, v_prev)

In [46]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(extended_gcd(240, 46))

(-9, 47)


In [47]:
# Your solution is not necessarily correct just because it passes these tests.

assert extended_gcd(240,46) == (-9, 47) # yes, from wikipedia ofc ;P

In [48]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.2 Calculating `d` (1 point)

Now that you have implemented the extended Euclidean algorithm, calculating `d` should be a breeze! Still, you decide to implement a function that returns `d` given `p`,`q`, and `e`. Make sure that it always returns a positive number. 

In [49]:
def calculate_d(p: int, q: int, e: int) -> int:
    # YOUR CODE HERE
    phi = (q-1)*(p-1)
    k,d = extended_gcd(e, phi)
    d = d%phi
    return d

In [50]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(calculate_d(5, 11, 3))

27


In [51]:
# Your solution is not necessarily correct just because it passes this test.
assert calculate_d(5, 11, 3) == 27

In [52]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.3 What is public and what is private? (1 point)

Prompty McBug has looked at your code and nods along, but you can quickly tell that he has no clue. After a while, you get him to admit that he does not even know what RSA is. So you start explaining the concept of public and private keys, but he still is not able to map all the variables to the public and the private parts. Seems like you need to become as explicit as possible:

For each variable `p`, `q`, `N = p*q`, `e`, `d`, `phi_N = (p-1) * (q-1)` **state** whether the variable is considered to be public (i.e., can safely be shared) or private (i.e., must not be shared). Additionally, justify your answer for `phi_N`.

q = private

p = private

N = public

e = public

d = private

phi_N = private (nowing phi_N immediately allows an attacker to calculate the private key d)

### Task 1.4 Encryption and decryption  (1 point)

Seems like you have convinces Promty that you know what you are doing. So, let's finally finish the RSA implementation, and implement a function `rsa_enc` to encrypt messages, and a function `rsa_dec` to decrypt messages.

Regarding the encryption, the public key is a tuple `(e, N)`, where both `e` and `N` are integers. The argument `message` represents the message to be encrypted using the given key. This message is in an integer format and is not a human-readable string. Just assume it to be some message as an integer and process it as discussed in the lecture. Further, it always holds that $message < N$. This function has to encrypt the given `message` with the given public key `e` and `N`.

The function `rsa_dec(cipher: int, private_key: tuple[int, int]) -> int` works similarly, but has to decrypt the message against using the private key. The private key is represented by the tuple `(d, N)`. The encrypted message is delivered as the argument `cipher`.

In [53]:
def rsa_enc(message: int, public_key: tuple[int, int]) -> int:
    e, N = public_key
    # YOUR CODE HERE
    return (message**e)%N

In [54]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.
print(rsa_enc(51, (7, 77)))

72


In [55]:
# Your solution is not necessarily correct just because it passes this test.
assert rsa_enc(51, (7, 77)) == 72

In [56]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [57]:
def rsa_dec(cipher: int, private_key: tuple[int, int]) -> int:
    d, N = private_key
    # YOUR CODE HERE
    return pow(cipher, d) % N

In [58]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.
print(rsa_dec(72, (43, 77)))

51


In [59]:
# Your solution is not necessarily correct just because it passes this test.
assert rsa_dec(72, (43, 77)) == 51

In [60]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

## 2. Efficient RSA (3 points)

Satisfied with your work, you decide to grab your favorite caffeinated drink. Big mistake! In these five minutes, Prompty McBug has instructed his trustworthy LLM to do a quick refactor and to push to prod. Astonishingly, the sloplock's CI/CD pipeline runs extremely smoothly, and the first customer complaints arrive by the time you are back at your desk:

* "The door takes like forever too unlock after the last update" - 1 star, anonymous review
* "WTF did the last update do? The pizza delivery guy needed to wait outside the locked door for 5 minutes! He almost left!" - 1 star, anonymous review
* "If you also have issues with opening the door, just download this app instead: robbers.nearby.example/sloplock-phishing-app" - 5 stars, anonymous review
* "The app is very slow, and the battery is empty after opening the door three times. Would give 0 stars if I want to" - 1 star, anonymous review
* "I like the new update" - 5 star, reviewer: "We only sell batteries GmbH & Co. KG"

These kind of reviews continue to pour in while you sift through Prompty's "small" 9000-deletions, 10000-additions refactor commit. After hours of draining work, you find that the LLM has introduced the following helper method:

In [61]:
def exp_mod(base: int, exponent: int, modulus: int) -> int:
    # Fast exponentiation modulo a prime, we keep all intermediary values modulo the modulus.
    result = 1
    for _ in range(exponent):
        result = (base * result) % modulus
    return result

While the comment claims to speed up the exponentiation, as all intermediary values are kept modulo the modulus, you notice that it has a runtime of `O(exponent)`. This is horrendous when you have really large exponents! In fact, the only reason why the users were able to open their doors at all (instead of computing exponents until the heat-death of the universe) was because the refactor changed the key sizes to be ridiculously small. Having found the root issue of the problem, your task is now to actually speed up the exponentiation.

**Implement** the square-and-multiply algorithm presented on Slide 22 of Chapter 4. **You must not use the Python-builtin `pow` function.**

In [62]:
def fast_exp_mod(base: int, exponent: int, modulus: int) -> int:
    # YOUR CODE HERE
    result = 1
    base = base % modulus

    if modulus == 1:
        return 0

    if base == 0 and exponent > 0:
        return 0
    
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus

        base = (base * base) % modulus
        exponent = exponent // 2
        
    return result

In [63]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.
import time
start = time.time()
print(fast_exp_mod(72, 43, 77))
print(fast_exp_mod(2, 2**16+1, 73421))
print(fast_exp_mod(10025419933826138, 1002541993382324, 100254199338261387))
stop = time.time()

print(f"The cell ran in {stop-start:.2f} seconds. On our grading system, the runtime of our intended solution is rounded to 0.00 seconds.")

51
3078
32639515412467585
The cell ran in 0.00 seconds. On our grading system, the runtime of our intended solution is rounded to 0.00 seconds.


In [64]:
# Your solution is not necessarily correct just because it passes this test
assert fast_exp_mod(72, 43, 77) == 51

In [65]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

## 3. Backdoors on RSA (6.5 points)

You are pretty fed up by Promty McBug's antics, and almost consider quitting on the very first day of your job, when your old empoyer, the Agency from Exercise 2, reaches out to you. It seems like the current big bad evil guy has upgraded his hidden bunker to use Sloplocks. Introducing a backdoor in the Sloplocks would allow spies convienient entry. Thus, they ask you to add a backdoor to the locks encryption schemes.

### Task 3.1 A naïve approach (3.5 points)

Given Promty's incompetence, you decide that the best approach would be to introduce a backdoor that easily can be blamed on Promty, and that gives the whole SLOP-team some plausible deniability. Hence, you decide to fix one prime number `p` such that every generated key uses the same `p`, but a different prime `q`, and to sell that as a performance improvement.

#### Task 3.1.1 Key generation (1 point)

The first step to introducing that backdoor is to implement the **generate_key_with_naive_backdoor**-function, that accepts a 1024-bit prime as `fixed_p`, and returns a **2048-bit** key-pair `((e, N), (d, N))`, where `(e,N)` is the public key, and `(d, N)` is the private key. One of the prime-factors of `N` must be `fixed_p`.

In [66]:
# This cell ensures that you have efficient RSA-building blocks available (which were built in the previous Tasks).
# NOTE THAT using these functions in the previous tasks WILL NOT GET YOU ANY POINTS if the task asks you implement a specific algorithm, like the extended euclidean algorithm or the square-and-multiply algorithm.
# Also: If you're ever doing crypto-challenges in a CTF, note how useful pow can be. It truely is a powerhouse w.r.t. modulus arithmetric.

def generate_prime(bit_size: int) -> int:
    # Generates a prime of bit-length bit_size.
    # This implementation is extremely cursed. You can see it as a testament 
    # to lengths I go so that you do not have to install another library.
    from cryptography.hazmat.primitives.asymmetric.rsa import generate_private_key
    return generate_private_key(2**16+1, 2*bit_size).private_numbers().p

def rsa_enc(message: int, public_key: tuple[int, int]) -> int:
    e, N = public_key
    return pow(message, e, N)

def rsa_dec(cipher: int, private_key: tuple[int, int]) -> int:
    d, N = private_key
    return pow(cipher, d, N)

def calculate_d(p: int, q: int, e: int) -> int:
    return pow(e, -1, (p-1)*(q-1))

In [67]:
def generate_key_with_naive_backdoor(fixed_p: int) -> tuple[tuple[int, int], tuple[int, int]]:
    # You can assume that the bit-length of "fixed_p" is 1024 bits.
    # YOUR CODE HERE
    q = generate_prime(1024)
    N = q * fixed_p
    e = get_e(fixed_p,q)
    d = calculate_d(fixed_p, q, e)
    return ((e,N),(d,N))

In [68]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.
fixed_p = generate_prime(1024)
public, private = generate_key_with_naive_backdoor(fixed_p)
print(public, private)

print(rsa_enc(rsa_dec(1234, public), private))

(5, 31013599116639617291938528553216645598683516784879035658567499942277323278727469057238546484911594190971394426006531873010257834571413843656391234684088653360338721448544767810398566299432344340295634229415064334839869271159428120615109499268783677429029729463327027404827653159377570930789848787798396673865533078401200535031258930848874436510439196045847929344890849045833209176408471129309577317406865733985845662156564815767241856219073587522236580882640856723933238168756561220393245277034199199426284639934686302966061595788686764956695091417182310182343894687978786400846788217467517844001072407925595636118807) (2481087929331169383355082284257331647894681342790322852685399995382185862298197524579083718792927535277711554080522549840820626765713107492511298774727092268827097715883581424831885303954587547223650738353205146787189541692754249649208759941502694194322378357066162192386212252750205674463187903023871733909214464069870169296557807231014460864838119894992791035382935981450648

In [69]:
# Your code is not necessarily correct just because it passes this simple test case.
fixed_p = generate_prime(1024)
public, private = generate_key_with_naive_backdoor(fixed_p)
N = public[1]
assert (N % fixed_p) == 0
assert N.bit_length() == 2048
assert rsa_enc(rsa_dec(1234, public), private) == 1234
del public
del private
del fixed_p

In [70]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

#### Task 3.1.2 Extracting the `fixed_p` (1.5 points)

Because you want some plausible deniability, you decide not to simply tell the Agency your fixed p. Instead, you provide them with a script that will allow them to recover `fixed_p` on their own, given they have two public keys. Hence, it looks like the Agency broke the Sloplocks on their own, and you are not a spy, but simply a well-meaining programmer that just wanted to speed-up the key generation.

**Implement** the function `recover_fixed_p` that accepts to backdoored public keys as input and returns the `fixed_p` used to generate the keys.

In [71]:
def recover_fixed_p(public_key_1: tuple[int, int], public_key_2: tuple[int, int]) -> int:
    e1, N1 = public_key_1
    e2, N2 = public_key_2
    # YOU MUST NOT USE "fixed_p" IN HERE!
    # YOUR CODE HERE
    return gcd(N1,N2)
    

In [72]:
### PLAYGROUND

fixed_p = generate_prime(1024)
print(f"Fixed p: {fixed_p}")
pub1, _ = generate_key_with_naive_backdoor(fixed_p)
pub2, _ = generate_key_with_naive_backdoor(fixed_p)

recovered_p = recover_fixed_p(pub1, pub2)
print(f"Recovered p: {recovered_p}")
del pub1
del pub2
del fixed_p

Fixed p: 151228584583240336424619247429826957620194557077130971516566357319738910618938911553319072396095120590375735745037705740011229172461238782133449401774891890176849597707073063967839725173834366874734945548640675419274469674788175750092375538903982901357039850336648742981758070484981269779092177793710425149479
Recovered p: 151228584583240336424619247429826957620194557077130971516566357319738910618938911553319072396095120590375735745037705740011229172461238782133449401774891890176849597707073063967839725173834366874734945548640675419274469674788175750092375538903982901357039850336648742981758070484981269779092177793710425149479


In [73]:
# Your code is not necessarily correct just because it passes this simple test case.
fixed_p_public_test = generate_prime(1024)
pub1, _ = generate_key_with_naive_backdoor(fixed_p_public_test)
pub2, _ = generate_key_with_naive_backdoor(fixed_p_public_test)
recovered_p = recover_fixed_p(pub1, pub2)
assert recovered_p == fixed_p_public_test
del pub1
del pub2
del fixed_p_public_test

In [74]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

#### Task 3.1.3 Decrypting messages (1 point)

Now that the `fixed_p` can be extracted, the final missing piece the actual decryption of messages. Implement the function `decrypt_naive_backdoor`, which accepts a ciphertext `cipher`, a backdoored public key `public_key = (e, N)`, and the recovered `fixed_p` as input. Return the decrypted message as integer.

*Hint: You need to compute the `d` of the private key.*

In [75]:
def decrypt_naive_backdoor(cipher: int, public_key: tuple[int, int], fixed_p: int) -> int:
    e, N = public_key
    # YOUR CODE HERE
    q = N // fixed_p
    d = calculate_d(fixed_p, q, e)
    return rsa_dec(cipher, (d,N))

In [76]:
### PLAYGROUND

fixed_p = generate_prime(1024)
pub1, _ = generate_key_with_naive_backdoor(fixed_p)
cipher = rsa_enc(1234, pub1)
print(decrypt_naive_backdoor(cipher, pub1, fixed_p))

1234


In [77]:
# Your solution is not necessary correct just because it passes this simple test.
fixed_p = generate_prime(1024)
pub1, _ = generate_key_with_naive_backdoor(fixed_p)
cipher = pow(1234, pub1[0], pub1[1])
assert decrypt_naive_backdoor(cipher, pub1, fixed_p) == 1234

pub2, _ = generate_key_with_naive_backdoor(fixed_p)
pub3, _ = generate_key_with_naive_backdoor(fixed_p)

recovered_p = recover_fixed_p(pub1, pub2)
cipher = pow(1234, pub3[0], pub3[1])
assert decrypt_naive_backdoor(cipher, pub3, recovered_p) == 1234

del fixed_p
del recovered_p
del pub1
del pub2
del pub3
del cipher

In [78]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 3.2 An advanced backdoor (3 points)

Using the backdoor, the good spy was able to infiltrate the bunker and evade security personel for a while. He was caught off-guard at some point, and had to endure a overly lengthy monologue explaining the big bad evil guy's plan to destroy the world. Still, he was able to escape the following pit of dinosaurs with lazers, and prevented a nuclear catastrophe. In the final mission briefing he remarked that the Sloplocks were really convinient, and he wishes that our Agency also would have these great modern locks.

Despite your pleas to burn everything that touches a Sloplock to the ground, the Agency listens to the spy and decides to install Sloplocks everywhere. Of course, the Sloplocks installed by the Agency must remain secure, but the Agency also really likes to have a backdoor in other peoples' Sloplocks. Hence, a redesign of the backdoor is required, such that only the Agency is able to use the backdoor. It is very important that no one else gets to use the backdoor. so you do not even need to be subtle anymore and the plausible deniability idea is thrown out the window.

The new backdoor design you come up with works as follows:
1. The agency generates a RSA-keypair (using the cell below).
2. When generating a backdoored sloplock key, you encrypt one of the primes against the Agency's public key, and use the resulting value as `e`.
3. To decrypt a message (encrypted against a backdoored key), the Agency can decrypt `e` to obtain the factorization of the backdoored public key.

Since the Agency's private key is needed to decrypt `e` into one of the prime factors, only the Agency is able to use the backdoor. Thus the Agency is safe from unwanted intruders, while the good spies are free to intrude the big bad evil bunkers.

In [79]:
def generate_agency_keypair() -> tuple[tuple[int,int],tuple[int,int]]:
    from cryptography.hazmat.primitives.asymmetric.rsa import generate_private_key
    private_key = generate_private_key(2**16+1, 2040) # Not 2048 on purpose.
    public_key = private_key.public_key().public_numbers()

    return ((public_key.e, public_key.n), (private_key.private_numbers().d, public_key.n))

agency_public_key, agency_private_key = generate_agency_keypair()

#### Task 3.2.1. Generating back-doored keys (2 points)

**Tasks 3.2.1 and 3.2.2 share the submission code cell below to allow nicer public tests.**

The first step to implementing the backdoor is to implement the backdoored key generation in the function `generate_key_with_advanced_backdoor`. This function accepts the public key of the Agency as input as integer tuple `(e, N)`. It should then sample two random 1024 bit primes, and encrypt one of them using the Agency's public key to obtain an `e`. Note that this might take multiple attempts, as not every `e` obtained through the encryption is a suitable `e` for the public key.

#### Task 3.2.1. Decrypting messages (1 point)

The second step is to decrypt messages encrypted against a backdoored public key. To that end, implement the function `decrypt_advanced_backdoor` that takes a ciphertext, the backdoored public key, and the Agency's private key, and that returns the decrypted message as int.

In [80]:
def generate_key_with_advanced_backdoor(agency_public_key: tuple[int, int]) -> tuple[tuple[int, int], tuple[int, int]]:
    # YOUR CODE HERE
    e_agency, N_agency = agency_public_key
    p = generate_prime(1024) 
    q = generate_prime(1024)
    e = rsa_enc(p,agency_public_key)
    phi = (p-1)*(q-1)
    while gcd(e,phi)!=1:
        p = generate_prime(1024)
        e = rsa_enc(p,agency_public_key)
        phi = (p-1)*(q-1)
    N = p * q
    d = calculate_d(p, q, e)
    return ((e,N), (d,N))

def decrypt_advanced_backdoor(cipher: int, backdoored_public_key: tuple[int, int], agency_private_key: tuple[int, int]) -> int:
    # YOUR CODE HERE
    e, N = backdoored_public_key
    p = rsa_dec(e, agency_private_key)
    q = N // p
    d = calculate_d(p, q, e)
    return rsa_dec(cipher, (d, N))

In [81]:
### PLAYGROUND

pub, private = generate_key_with_advanced_backdoor(agency_public_key)
cipher = rsa_enc(1234, pub)
print(f"Decrypted with real private key: {rsa_dec(cipher, private)}")
print(f"Decrypted with agency private key: {decrypt_advanced_backdoor(cipher, pub, agency_private_key)}")

Decrypted with real private key: 1234
Decrypted with agency private key: 1234


In [82]:
# Your code is not neccessarily correct just because it passes this simple test.
pub, private = generate_key_with_advanced_backdoor(agency_public_key)
cipher = rsa_enc(1234, pub)
assert rsa_dec(cipher, private) == 1234
del private # The Agency does not know the private key (yet).
assert decrypt_advanced_backdoor(cipher, pub, agency_private_key) == 1234

In [83]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [84]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

## 4. General Questions (5 points)

This task is now focused on written answers regarding RSA and asymmetric cryptography.

### Task 4.1 - Advantages and Disadvantages of Asymmetric Cryptography (1 point)

Think about asymmetric cryptography in general. There are quite a lot of differences in comparison to symmetric cryptography. However, both are being used for encryption and integrity protection.

**Name** at least one advantage and one disadvantage of asymmetric cryptography in regard to encrypting messages and authenticating them.

Key distribution is easier and more secure (public key can be shared openly).

Computationally much slower than symmetric cryptography.

### Task 4.2 - Digital Signatures and Forgery (2 points)

Asymmetric cryptography is often used to create signatures on messages or other important information. However, these signatures can also be attacked. There are different kinds of results of attacks.

**Name** two different results of attacks on digital signatures and **explain** what the attacker is able to do.
*Hint: Make sure that your answer is precise, i.e., that the attacker's capabilities are clearly differentiated from other attacks on digital signatures.*

Total Break: (Partial) recovery of the signature key.

Selective Forgery: Creates a valid signature for a specific message chosen before the attack is initiated, again without knowing the private key. Only able to forge for some messages.

### Task 4.3 - RSA Signatures (2 points)

Now, we consider RSA signatures in general. 
Why is the hash of the message instead of the message itself signed? **Name** two reasons.

Efficiency: Signing the full message is computationally slow, especially for long messages. Hashing provides a fixed-length which makes the RSA signature operation much faster.

preimage resitance: As hash functions are preimage resistant, an attacker would not be able to forge a message with h(m) = s^e
                    --> the attacker is not able to forge a message with a predefined hash value

## 5. Make sure that your code is working (0 points)

Please make sure that your code is working, i.e., check whether
- you import only built-in Python libraries and libraries provided by us, ideally, you should not have written any import yourself,
- your code is compatible with Python 3.12,
- restarting the Jupyter kernel and re-running all code cells works without errors,
- all assertions provided by us pass.

Technically, this is not a task, just a gentle reminder to make sure that your previous solutions are working :)

## 6. Feedback (0.5 points)

You made it through another assignment! Since we want to know how it went and how we might improve the exercises, we include the following task. Here, you can write constructive feedback! You even get 0.5 points for it if you write anything. But don't worry, we do not grade the content itself!

Very nice structure, story was funny. Kinda uncool to just copy lecture slides.

Nevertheless it was sometimes confusing when someone forgot to run every single prewritten codeline.

9/10 homework experience.